# Unsupervised anomaly detection with Temporian and scikit-learn

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google/temporian/blob/last-release/docs/src/tutorials/anomaly_detection_unsupervised.ipynb)

In this tutorial, we use Temporian and Scikit-Learn to detect anomalies in a multivariate time series dataset.

Anomaly detection in time series, time sequences and other temporal formats is critical in a variety of domains. For instance, it is used in manufacturing to detect equipment failure and production lines, in computer systems and by financial institutions to detect fraudulent activities, and in energy management to detect outages.

We will use the Server Machine Dataset (SMD) dataset, published as part of the [OmniAnomaly](https://github.com/NetManAIOps/OmniAnomaly/) paper, which is available in CSV files in that same repository.

This dataset contains aggregated resource usage metrics (e.g., CPU and RAM utilization, network traffic) from 28 computers in 3 data centers over a period of 5 weeks. The timestamps and feature names have been anonymized and normalized. Therefore, calendar feature engineering or the use of expert knowledge is not possible.

The dataset will be loaded, feature engineered and converted into a tabular dataset using Temporian. Then, we will use an isolation forest model on this tabular data to detect anomalies. Finally, we will evaluate the quality of our detection on the ground truth anomalies, available as part of the testing dataset.

We will first evaluate our detections both with AUC-[ROCs](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) (Area Under the Receiver operating Characteristic curve) and with [AMOCs](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2815453/) (Activity Monitoring Operating Characteristic). Unlike an ROC, an AMOC takes into account the *time to detection* or *forecasting horizon* which is often critical in temporal problems like ours. If you don't know about AMOCs yet, don't worry. We will explain it.

Check out the [Supervised anomaly detection](https://temporian.readthedocs.io/en/stable/tutorials/anomaly_detection_supervised/) tutorial for a version of this notebook that trains a supervised model using the ground truth labels, which is less common in an anomaly detection setting, but can yield better performance if those are available.

## Installation and imports

In [ ]:
%pip install temporian -q

In [ ]:
import os
from pathlib import Path
from typing import List

import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.metrics import roc_auc_score
import urllib.request

import temporian as tp

## Download dataset

The dataset is made of 3 groups of 8, 9, and 11 machines respectively, with names `"machine-1-1"`, ..., `"machine-3-11"`.
Let's list the machine names, and download records locally.

In [ ]:
# machines_per_group = [8, 9, 11]  # Full dataset
machines_per_group = [3, 3, 3]  # Subset of data

machines = [f"machine-{group}-{id}" for group, machine in zip(range(1, 4), machines_per_group) for id in range(1, machine + 1)]
print("{len(machines)} machines")

In [ ]:
data_dir = Path("tmp/temporian_server_machine_dataset")
dataset_url = "https://raw.githubusercontent.com/NetManAIOps/OmniAnomaly/master/ServerMachineDataset"

data_dir.mkdir(parents=True, exist_ok=True)

# Download the data and labels for each machine to its own folder
for machine in machines:
    print(f"Download data of {machine}")

    machine_dir = data_dir / machine
    machine_dir.mkdir(exist_ok=True)

    data_path = machine_dir / "data.csv"
    if not data_path.exists():
        urllib.request.urlretrieve(f"{dataset_url}/test/{machine}.txt", data_path)

    labels_path = machine_dir / "labels.csv"
    if not labels_path.exists():
         urllib.request.urlretrieve(f"{dataset_url}/test_label/{machine}.txt", labels_path)

## Load data

We use Pandas to load the data into a `DataFrame` and perform tabular manipulation before importing it into a Temporian as an `EventSet`.

We index the data by the `machine` column in Temporian. This was, we can apply Temporian transformations on each machine individually as well as across all machines.

In [ ]:
dataframes = []

for machine in machines:
    machine_dir = data_dir / machine

    # Read the data and labels
    print(f"Load data of {machine}...", end="")
    df = pd.read_csv(machine_dir / "data.csv", header=None).add_prefix("f")
    labels = pd.read_csv(machine_dir/ "labels.csv", header=None)
    df = df.assign(label=labels)
    df["machine"] = machine
    df["timestamp"] = range(df.shape[0])
    print(f"found {df.shape[0]} events")

    dataframes.append(df)

dataframes[0].head(3)

In [ ]:
# Convert the dataframes into a single Temporian EventSet

evset = tp.combine(*map(tp.from_pandas, dataframes))

# Index the EventSet according the the machine name.
evset = evset.set_index("machine")

# Cast the feature and label to a smaller dtypes to same one memory.
evset = evset.cast(tp.float32).cast({"label": tp.int32})

evset

Awesome! Seems like each machine has more than 20.000 events and 39 features (counting the "label" one).

As stated previously, all metrics seem to be anonymized and normalized to `[0, 1]`, so we won't need to take care of that ourselves.

## Data visualization

Let's take a look at some of the first machine's features.

In [ ]:
# Plot the first 3 features
evset.plot(indexes="machine-1-1", max_num_plots=3)

# Plot the labels
evset["label"].plot(indexes="machine-1-1")

Great! A lot to unpack here:
- It seems to be easy to understand when an anomaly occurs (label takes value of 1) by looking at the other plots. Features 11 to 14, for example, seem to be very correlated to the label.
- The data seems to have some periodicity to it.
- Some features seem empty, and we could evaluate dropping them if needed.

## Data preparation

To prepare our data to train a model on it, let's start off by separating the features from the labels.

In [ ]:
feature_names = evset.schema.feature_names()
feature_names.remove("label")

raw_features = evset[feature_names]
labels = evset["label"]

print("Raw features:", raw_features.schema)
print("Labels:", labels.schema)

Next, we'll need to split our dataset into train and testing sets, which we'll use an 80/20 split for.

Note that the train labels will only be used for evaluation purposes.

We'll be creating reusable functions for each step, since we'll do some iteration over the `feature engineering -> training -> evaluation` cycle.

In [ ]:
DROP_COLS = ["machine", "timestamp"]

def split_event_set(evset: tp.EventSet, test_frac: float = 0.2):

    # Average length of the records
    average_duration = np.mean([len(machine_data.timestamps) for _, machine_data in evset.data.items()])
    print("average_duration:", average_duration)

    # Select the train/test cutoff.
    # Note: All the machines are cut at the same time. This way, we can apply pre-processing that
    # exchange data between the machines without risk of label leakage!
    train_cutoff = average_duration * (1 - test_frac)
    print("train_cutoff:", train_cutoff)

    # Compute masks and split data based on cutoff
    train_mask = evset.timestamps() <= int(train_cutoff)
    test_mask = ~train_mask

    # Split EventSets
    train_evset = evset.filter(train_mask)
    test_evset = evset.filter(test_mask)

    print(f"Train events: {train_evset.num_events()}")
    print(f"Test events: {test_evset.num_events()}")

    return train_evset, test_evset

def evsets_to_tabular(*evsets: tp.EventSet) -> List[np.ndarray]:
    datasets = []
    for evset in evsets:
        # Fill missing values with -1 (raw data doesn't have any, but we will create some in our feature engineering)
        df = tp.to_pandas(evset).fillna(-1)

        # Remove timestamp and machine columns
        df = df.drop(columns=DROP_COLS)

        # Convert to numpy (and convert from 2D to 1D array if it has a single feature, in the case of the labels)
        arr = df.to_numpy().squeeze()

        datasets.append(arr)

    return datasets

features_train, features_test = split_event_set(raw_features)
labels_train, labels_test = split_event_set(labels)  # Note that we won't be using the train labels for training in this unsupervised setting

X_train, X_test, y_train, y_test = evsets_to_tabular(features_train, features_test, labels_train, labels_test)

## Training

Having done all that work to prepare our data, all that remains is to train our model.

In [ ]:
contamination = y_train.sum() / len(y_train)
print(f"{contamination=}")

def train(X_train):
    model = IsolationForest(
        contamination=contamination,
        n_jobs=-1,
        verbose=1,
        random_state=0,
    )
    model.fit(X_train)
    return model

In [ ]:
model = train(X_train)

## Evaluation using AUC-ROC

We'll be reporting the model's ROC AUC score, which provides an aggregate measure of performance across all possible classification thresholds (since our model outputs an anomaly score for each sample, and in a real-world scenario it would be up to us to define the thershold from which we consider an event to be marked as anomalous).

In [ ]:
figsize=(20,3)

results = {}

def evaluate(model, X_test, y_test, name):
    """Evaluates a model on its training data and unseen test data, computing accuracy score and plotting ground truth vs predictions."""
    # Compute and print scores
    roc_auc = roc_auc_score(y_test, -model.score_samples(X_test))

    results[name] = {"ROC AUC": roc_auc}

    print("Results:")
    print(pd.DataFrame(results))

    # Plot predictions
    preds = model.predict(X_test)
    preds[preds == 1] = 0
    preds[preds == -1] = 1
    tp.event_set(timestamps=list(range(len(y_test))), features={"labels": y_test, "predictions": preds}).plot(style="vline")

In [ ]:
evaluate(model, X_test, y_test, "raw features")

That's pretty decent for a first try! Our model seems to be learning.

The plotted predictions seem to be all over the place though, with the model sporadically predicting anomaly events throughout in several non-anomalous periods.

There's plenty of room for improvement, so let's kick off the feature engineering!

## Evaluation using AMOC

An AMOC measure the relation between false alerts (or average time between false alerts) and the average time to detection.
Using Temporian, it is easy to implement an AMOC.

Here is how an AMOC is computed.

We want to detect "anomalies" events. In our dataset, we generate an anomaly event each time the "label" (or "target") is true.
Our model creates a numerical detection "signal". Like for an AUC, we evaluate our model with different threshold values applied on the signal.
An "alert" event is generated each time the "signal" goes over the detection threshold.


A "false alert" is an alert that does not detect an anomaly event i.e. an alert a`t` time t without any anom`aly in [t-abnomaly_win`dow, t].
A "missed anomaly" is an anomaly that is not detected by any alert i.e. an anomaly `a`t time t without any a`lert in [t, t+abnomaly`_window].
The "time to detection" of an anomaly` `at time t is the time between the anomaly and the first following `alert in [t, t+abnomal`y_window]. If there is not alert in this time window, the "time to detection" is assumed to be abnomaly_window.

In practice, two anomalies that happen next to each other in time can be considered the same anomaly. So we apply a "shutoff" on the anomalies:
Two anomalies cannot happen within "missed_target_shutoff" of each other.
Similarly, we apply a "shutoff" on the false alerts.

Here is a diagram to summarise

![](https://raw.githubusercontent.com/google/temporian/main/docs/src/assets/amoc.svg)

An AMOC can be computed on any signal, not just a model output. Let's compute the AMOC on the features `f1` and `f2`.
We can find which one of the two features alone can best detect anomalies.he situation:



In [ ]:
from dataclasses import dataclass
from typing import Optional, Dict, List
import matplotlib.pyplot as plt

@dataclass
class ThresholdEval:
    """Model metric for each threshold value applied on the score."""

    threshold: float
    num_false_alerts: int
    num_missed_anomalies: int
    average_time_to_detection: float
    total_record_duration: float
    num_machines: int

def compute_detection_amoc(target : tp.EventSet,
                   signal : tp.EventSet,
                   detection_thresholds: List[float] = np.linspace(0,1,100, dtype=np.float32).tolist(),
                   anomaly_window : float = 1000.,
                   anomaly_shutoff : float = 200.,
                   false_alert_shutoff : float = 200.,
                   plot_first: bool = True,
                  ) -> List[ThresholdEval]:
    """Computes a detection AMOC.

    Args:
        target: A boolean feature defining an anomaly to detect.
        signal: A numerical feature detecting recent anomalies. A signal above a threshold becomes an alert.
        detection_thresholds: List of evaluated detection thresholds applied on the "signal".
        anomaly_window: For how long an anomaly can be detected.
        anomaly_shutoff: Minimum distance between two anomalies.
        false_alert_shutoff: Minimum distance between two missed alerts.
        plot_first: Plot the alerts, anomalies of the first index and the mid-threshold.
    """

    # List the anomalies
    anomalies = target.cast(tp.bool_).filter()
    anomalies = anomalies.filter_moving_count(anomaly_shutoff)

    # Final timestamp used to collect all the metrics.
    end_of_records = signal.end()
    end_of_records_global = end_of_records.drop_index(keep=False).end()

    def sum_by_index(evtset) -> int:
        """Sums of the last value of a feature in all the indexes."""
        assert len(evtset.schema.features) == 1 # There should be only one feature

        return float(evtset.drop_index(keep=False).cumsum(sampling=end_of_records_global).get_arbitrary_index_data().features[0][0])

    # The duration of record in each index is the time distance between the first and last event.
    record_duration = signal.begin().since_last(sampling=signal.end())
    total_record_duration = sum_by_index(record_duration)
    print("total_record_duration:",total_record_duration)

    # There is one index per machine
    num_machines = len(target.data)
    print("num_machines:",num_machines)

    num_anomalies = int(anomalies.drop_index(keep=False).moving_count(np.inf, sampling=end_of_records_global).get_arbitrary_index_data().features[0][0])
    print("num_anomalies:",num_anomalies)

    # Can also be computed this way:
    assert num_anomalies == np.sum([len(machine.timestamps) for key, machine in anomalies.data.items()])

    amoc = []

    for threshold_idx, threshold in enumerate(detection_thresholds):
        is_mid_threshold = threshold_idx == (len(detection_thresholds)//2)

        alerts = (signal >= threshold).filter()[[]]

        # False alerts
        false_alerts = tp.equal_scalar(anomalies.moving_count(anomaly_window, sampling=alerts), 0).filter()
        false_alerts = false_alerts.filter_moving_count(false_alert_shutoff)
        num_false_alerts = false_alerts.moving_count(np.inf, sampling=end_of_records)

        # Missed targets
        missed_anomalies = tp.equal_scalar(alerts.moving_count(anomaly_window, sampling=anomalies.lag(anomaly_window)), 0).filter().leak(anomaly_window)
        missed_anomalies = missed_anomalies.filter_moving_count(false_alert_shutoff)
        num_missed_anomalies = missed_anomalies.moving_count(np.inf, sampling=end_of_records)

        # Time to detection
        time_to_detection = anomalies.until_next(sampling=alerts, timeout=anomaly_window)
        # Set the time to detection of non detected anomalies with the maximum anomaly_window.
        time_to_detection = tp.isnan(time_to_detection).where(anomaly_window, time_to_detection)
        # Note: Some machines don't have alerts
        sum_time_to_detection = time_to_detection.cumsum(sampling=end_of_records)

        if plot_first and is_mid_threshold:
            anomalies.name = "anomalies"
            alerts.name = "alerts"
            missed_anomalies.name = "missed_anomalies"
            false_alerts.name = "false_alerts"
            tp.plot([
                target,
                signal,
                anomalies,
                alerts,
                missed_anomalies,
                false_alerts
                ],
                    indexes="machine-1-1",
                   height_per_plot_px=100)

        amoc.append(ThresholdEval(
            threshold=threshold,
            num_false_alerts=sum_by_index(num_false_alerts) / num_machines,
            num_missed_anomalies=sum_by_index(num_missed_anomalies) / num_machines,
            average_time_to_detection=sum_by_index(sum_time_to_detection) / num_anomalies,
            total_record_duration=total_record_duration / num_machines,
            num_machines=num_machines,
        ))

    return amoc

def plot_detection_amoc(amoc_dict: Dict[str, List[ThresholdEval]]):

    # Plot the AMOC and other related plots
    fig, axs = plt.subplots(1, 2, figsize=(5*2, 3), squeeze=False)

    for name, amoc in amoc_dict.items():
        ax = axs[0,0]
        ax.plot([e.average_time_to_detection for e in amoc],
                [e.num_false_alerts for e in amoc],
               label=name)

        ax = axs[0,1]
        ax.plot([e.average_time_to_detection for e in amoc if e.num_false_alerts != 0],
                [e.total_record_duration / e.num_false_alerts for e in amoc if e.num_false_alerts != 0],
               label=name)

    ax = axs[0,0]
    ax.set_xlabel("Avg. time to detection")
    ax.set_ylabel("Num. false alerts")
    ax.legend()
    ax.grid(True)

    ax = axs[0,1]
    ax.set_xlabel("Avg. time to detection")
    ax.set_ylabel("Avg. time between false alerts")
    ax.set_yscale("log")
    ax.invert_xaxis()
    ax.legend()
    ax.grid(True)

amoc_f1 = compute_detection_amoc(evset["label"], evset["f1"])
amoc_f2 = compute_detection_amoc(evset["label"], evset["f2"], plot_first=False)

plot_detection_amoc({
    "f1":amoc_f1,
    "f2":amoc_f2,
})

We can see that both features performs relatively equally.
For instance, with an average time to detection of 400 time-units, both features generates approximatively ~30 false alerts for each machine.

What about our detection model now?

In [ ]:
def model_prediction_to_evtset(model):
    df = tp.to_pandas(features_test)[["machine", "timestamp"]]
    df["detection"] = -model.score_samples(X_test)
    df["label"] = y_test
    evtset = tp.from_pandas(df).add_index("machine")
    return evtset

pred_evtset = model_prediction_to_evtset(model)
amoc_model_raw_features = compute_detection_amoc(pred_evtset["label"], pred_evtset["detection"])

plot_detection_amoc({
    "f1":amoc_f1,
    "f2":amoc_f2,
    "model:raw features": amoc_model_raw_features,
})

The model is **much** better than the features alone. This is expected, but it is a good check to do :).

## Feature engineering

### Lag features

Right now our model only has access to each event's raw metric values, + the group and machine that it belongs to. This means that it has no knowledge of the **context** an event is happening on - some values might have been completely normal when the measuring started, but anomalous a couple of weeks later, e.g. if that machine's usage went up as a whole during that time and its baseline usage now stands much higher than it used to.

To combat this, we'll start by lagging the values of each feature. In doing this, we're providing the model (some) information about that the metric's value looked like a couple of steps into the past.

In [ ]:
lag_features = []

# Lag each raw feature by 1, 2, ..., 10 steps
for window in range(1, 11):
    lag_features.append(raw_features.lag(window).resample(raw_features).prefix(f"lag_{window}_"))

features = tp.glue(raw_features, *lag_features)
features

Let's take a look at any metric alongside its lagged values. We'll select a small time window, to be able to appreciate how the time series moves to the right as the number of lagged timesteps increases.

In [ ]:
f13_lags = features[["f13"] + [f"lag_{i}_f13" for i in range(1, 11)]]
timestamps = f13_lags.timestamps()
f13_lags = f13_lags.filter((timestamps > 21500) & (timestamps < 21600))
f13_lags.plot(max_num_plots=4)

Time to train and evaluate a new model with these new features! 

In [ ]:
features_train, features_test = split_event_set(features)
_, labels_test = split_event_set(labels)

X_train, X_test, y_test = evsets_to_tabular(features_train, features_test, labels_test)

model = train(X_train)
evaluate(model, X_test, y_test, "lagged features")

Nice! The lagged features seem to have helped the model quite a bit. The ROC AUC went up, and the predictions are already looking more solid, with several anomalous periods being correctly identified.

What about the AMOC?

In [ ]:
pred_evtset = model_prediction_to_evtset(model)
amoc_model_lagged_features = compute_detection_amoc(pred_evtset["label"], pred_evtset["detection"])

plot_detection_amoc({
    "model:raw features": amoc_model_raw_features,
    "model:lagged features": amoc_model_lagged_features,
})

**TODO:** The model with raw features is better.

### Moving statistic features

Although useful, the raw lagged values aren't enough to provide the model a comprehensive look at each value's past context. Note also that we only gave it a glimpse of 10 steps into the past, and each time series has more than 23k values.

This is where **moving statistics** can come in handy. Instead of a list of raw values, we can provide the model an aggregation of each metric's values over the last N timesteps. For example, we can tell it what the maximum and minimum value of a metric were in the last 20 steps, or what the standard deviation was in the last 1000. 

Luckily, Temporian's window operators make this a breeze.

In [ ]:
moving_statistic_features = []

# Compute the moving average, standard deviation, max, and min over different windows
for window in [20, 200, 2000]:
    moving_statistic_features.append(raw_features.simple_moving_average(window).prefix(f"avg_{window}_"))
    moving_statistic_features.append(raw_features.moving_standard_deviation(window).prefix(f"std_{window}_"))
    moving_statistic_features.append(raw_features.moving_max(window).prefix(f"max_{window}_"))
    moving_statistic_features.append(raw_features.moving_min(window).prefix(f"min_{window}_"))

features = tp.glue(raw_features, *lag_features, *moving_statistic_features)
features

Taking a look at some of the generated features:

In [ ]:
f13_stats = features[["f13", "avg_20_f13", "avg_200_f13", "max_20_f13", "max_200_f13"]]
timestamps = f13_stats.timestamps()
f13_stats = f13_stats.filter((timestamps > 21000) & (timestamps < 21100))
f13_stats.plot(indexes="machine-1-1")

Moving statistics can be a good indicator of what a sequence's "normal" behavior looks like. 

As an example, `"avg_20_f12"`'s plot shows how the average value of the series goes up with time - so it could allow the model to tell apart whether a value of `0.07` for `f13` is anomalous or not, depending on when it happens!

In [ ]:
features_train, features_test = split_event_set(features)
_, labels_test = split_event_set(labels)

X_train, X_test, y_test = evsets_to_tabular(features_train, features_test, labels_test)

model = train(X_train)
evaluate(model, X_test, y_test, "moving statistics")

That's quite an improvement! Moving statistics can be of inmense importance in an anomaly detection setting, since "anomalous" values tend to be defined as those that don't match the item's normal behavior, and moving statistics can help the model understand what "normal behavior" means for a a specific timeframe.

In [ ]:
pred_evtset = model_prediction_to_evtset(model)
amoc_model_moving_statistics = compute_detection_amoc(pred_evtset["label"], pred_evtset["detection"])

plot_detection_amoc({
    "model:raw features": amoc_model_raw_features,
    "model:lagged features": amoc_model_lagged_features,
    "model:moving statistics": amoc_model_moving_statistics,
})

**TODO:** The model with moving statsitics is event worst.

## Wrapping up

In this notebook we learned how to perform feature engineering and visualization using Temporian, applying it to a real-world anomaly detection use case.

There's some further work that could be done in this problem! Here's some ideas:
- Train a larger model! Training a larger model, on a larger dataset, would most likely yield improved results. As an example, try increasing the number of estimators in the IsolationForest (the default is 100), and optionally also increase the number of machines in each group to pull data from, and re-run the experiments!
- Use the dataset's [unlabeled train data](https://github.com/NetManAIOps/OmniAnomaly/tree/master/ServerMachineDataset/train) to train the model in this unsupervised manner, and then use the data we used here to test it only.
- Keep adding new features! As we demonstrated, a very simple model can go a long way if the correct features are provided to it. This is where Temporian shines - check out the full list of operators in the [API Reference](https://temporian.readthedocs.io/en/stable/reference/#operators) for some inspiration!